In [1]:
import os
import time

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.dailydata import DailyData
from libs.utils import Utils
from libs.quotation import Quotation

In [13]:
date = time.strftime('%Y%m%d')
date

'20210622'

In [16]:
data = DailyData.load(dt='20210618')

In [17]:
data.statistic.shape

(3007, 4331, 10)

In [18]:
df = data.get_securities()
df

,name,zt_price,dt_price,ma5vpm,mcap,sum4,sum9,sum19,sum29,sum59
symbol,,,,,,,,,,
000001,平安银行,25.41,20.79,"238,082.74","4,482.73",92.95,215.17,456.82,693.12,"1,347.71"
000002,万 科Ａ,27.50,22.50,"275,995.30","2,429.39",100.01,229.90,497.44,769.73,"1,648.09"
000004,国华网安,23.01,18.83,"56,390.39",24.08,86.63,180.42,339.53,493.54,"1,023.32"
000005,ST星源,1.93,1.75,"32,646.26",19.47,7.36,16.81,34.55,53.42,124.05
000006,深振业Ａ,5.70,4.66,"21,741.42",69.93,20.65,46.73,99.50,153.36,314.75
...,...,...,...,...,...,...,...,...,...,...
688777,XD中控,107.20,71.46,"7,096.49",36.67,359.29,824.65,"1,718.31","2,518.52","4,969.76"
688788,科思科技,158.38,105.58,"2,598.06",22.97,525.29,"1,199.03","2,537.03","3,588.25","6,681.62"
688819,天能股份,48.46,32.30,"14,544.42",40.81,165.03,383.10,799.75,"1,205.87","2,583.00"


In [19]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [20]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport fabs
from libc.math cimport isnan

ctypedef np.float64_t DTYPE_t

@cython.boundscheck(False)
@cython.cdivision
@cython.wraparound(False)
cpdef compute_stats(DTYPE_t[:,:] ms, DTYPE_t[:,:] b, DTYPE_t[:,:] st, DTYPE_t[:,:] fs5p, Py_ssize_t time_lapse):
    
    # ms
    cdef Py_ssize_t idx_open = 0
    cdef Py_ssize_t idx_now = 2
    cdef Py_ssize_t idx_high = 3
    cdef Py_ssize_t idx_low = 4
    cdef Py_ssize_t idx_close = 1
    cdef Py_ssize_t idx_turnover = 5
    cdef Py_ssize_t idx_volume = 6
    
    # statistic
    cdef Py_ssize_t idx_zf = 0
    cdef Py_ssize_t idx_jj = 1
    cdef Py_ssize_t idx_lb = 2
    cdef Py_ssize_t idx_zs = 3
    cdef Py_ssize_t idx_tb = 4
    cdef Py_ssize_t idx_sum5 = 5
    cdef Py_ssize_t idx_sum10 = 6
    cdef Py_ssize_t idx_sum20 = 7
    cdef Py_ssize_t idx_sum30 = 8
    cdef Py_ssize_t idx_sum60 = 9
    
    cdef Py_ssize_t rows = ms.shape[0]
    cdef Py_ssize_t cols = ms.shape[1]
    
    for i in range(rows):
        
        # if ms[i,idx_open]:
        #     st[i,idx_zf] = 100*(ms[i,idx_now]/ms[i,idx_close]-1)
        if ms[i,idx_now]:
            st[i,idx_zf] = 100*(ms[i,idx_now]/ms[i,idx_close]-1)
        else:
            st[i,idx_zf] = np.nan
            
        if ms[i,idx_turnover]:
            st[i,idx_jj] = ms[i,idx_volume]/ms[i,idx_turnover]
            st[i,idx_zs] = 100*(ms[i,idx_now]/fs5p[i, 0]-1)
            if fabs(ms[i,idx_high] - b[i,0]) < 0.001:
                st[i,idx_tb] = 0.5
                if fabs(ms[i,idx_high] - ms[i,idx_now]) < 0.001:
                    st[i,idx_tb] = 1
            elif fabs(ms[i,idx_low] - b[i,1]) < 0.001:
                st[i,idx_tb] = -0.5
                if fabs(ms[i,idx_low] - ms[i,idx_now]) < 0.001:
                    st[i,idx_tb] = -1
                    
            
            st[i,idx_sum5] = (b[i, 4]+ms[i,idx_now])/5
            st[i,idx_sum10] = (b[i, 5]+ms[i,idx_now])/10
            st[i,idx_sum20] = (b[i, 6]+ms[i,idx_now])/20
            st[i,idx_sum30] = (b[i, 7]+ms[i,idx_now])/30
            st[i,idx_sum60] = (b[i, 8]+ms[i,idx_now])/60
                

        if b[i,2] and ms[i,idx_turnover]:
            st[i,idx_lb] = ms[i,idx_turnover]/time_lapse/b[i,2]
            

In [ ]:
symbols = Utils.get_running_symbols()
q = Quotation(symbols)

In [ ]:
idx = 325
await q.snapshot(array=data.snapshots[idx])

In [6]:
idx = 325

In [21]:
%%timeit
time_lapse = data.get_time_lapse(idx)
ma5pm_anchor_idx = data.get_ma5pm_anchor_idx(idx)
fs5p = data.snapshots[ma5pm_anchor_idx]
compute_stats(data.snapshots[idx], data.basics, data.statistic[idx], fs5p, time_lapse)

211 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
st = time.time()
for _ in range(len(data.check_points)):
    time_lapse = data.get_time_lapse(_)
    ma5pm_anchor_idx = data.get_ma5pm_anchor_idx(_)
    fs5p = data.snapshots[ma5pm_anchor_idx]
    compute_stats(data.snapshots[_], data.basics, data.statistic[_], fs5p, time_lapse)
et = time.time()
et - st

0.6377475261688232

In [ ]:
data.statistic[-1000,:,:]

In [ ]:
data.snapshots[133]

In [ ]:
data.save()

In [ ]:
import numpy as np
import random

In [ ]:
arr = np.array([random.randint(0,100) for x in range(12)])

In [ ]:
arr = arr.reshape(3, 4)

In [ ]:
arr[:,-1].argsort()

In [ ]:
type(float('nan'))

In [ ]:
type(math.nan)

In [ ]:
np.nan